베르누이 나이브 베이즈를 활용한 스팸 매일 분류  
이메일 제목과 레이블(스팸 여부) 데이터를 활용해 베르누이 나이브 베이즈 분류로 스팸 메일을 분류한다.

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# 베르누이 나이브 베이즈를 사용하기 위해 import 한다.
from sklearn.naive_bayes import BernoulliNB
# 데이터에 출현한 모든 단어 개수 만큼의 크기를 가진 벡터를 만들고 고정된 벡터로 표현하기 위해 import 한다.
from sklearn.feature_extraction.text import CountVectorizer

데이터 획득  
간단한 스팸 메일 분류를 위해 다음과 같이 이메일 제목과 스팸 메일 레이블이 붙어있는 데이터를 사용한다.  
email title: 이메일 제목, spam: 스팸 메일 여부(True => 스팸 메일, False => 스팸 메일 아님)

In [3]:
# 학습 데이터 준비
email_list = [
    {'email title' : 'free game only today', 'spam' : True},
    {'email title' : 'cheapest flight deal', 'spam' : True},
    {'email title' : 'limited time offer only today only today', 'spam' : True},
    {'email title' : 'today meeting schedule', 'spam' : False},
    {'email title' : 'your flight schedule attached', 'spam' : False},
    {'email title' : 'your credit card statement', 'spam' : False}
]
df = pd.DataFrame(email_list)
df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


In [4]:
# 학습 데이터 다듬기
# 사이킷런의 베르누이 나이브 베이즈 분류기는 숫자 데이터만 다루기 때문에 True와 False를 1과 0으로 치환한다.
df['label'] = df.spam.map({True: 1, False: 0})
df

,email title,spam,label
0,free game only today,True,1
1,cheapest flight deal,True,1
2,limited time offer only today only today,True,1
3,today meeting schedule,False,0
4,your flight schedule attached,False,0
5,your credit card statement,False,0


학습에 사용할 데이터와 레이블(답, 실제값, 타겟, 클래스)로 분리한다.

In [5]:
df_x = df['email title'] # 피쳐
df_x

0                        free game only today
1                        cheapest flight deal
2    limited time offer only today only today
3                      today meeting schedule
4               your flight schedule attached
5                  your credit card statement
Name: email title, dtype: object

In [6]:
df_y = df['label'] # 타겟
df_y

0    1
1    1
2    1
3    0
4    0
5    0
Name: label, dtype: int64

이메일 제목(email title)으로 학습을 진행하고 레이블은 label을 사용해서 스팸 메일 여부를 판단한다.

베르누이 나이브 베이즈의 입력 데이터는 고정된(동일한) 크기의 벡터이어야 한다. 

In [20]:
string = 'free game only today cheapest flight deal limited time offer only today only today today meeting schedule ' \
    'your flight schedule attached your credit card statement'
print(string)
string = set(string.split())
print(len(string))
print(string)
string = list(string)
string.sort()
print(string)

free game only today cheapest flight deal limited time offer only today only today today meeting schedule your flight schedule attached your credit card statement
17
{'flight', 'limited', 'game', 'time', 'cheapest', 'only', 'free', 'today', 'meeting', 'your', 'deal', 'statement', 'card', 'credit', 'attached', 'offer', 'schedule'}
['attached', 'card', 'cheapest', 'credit', 'deal', 'flight', 'free', 'game', 'limited', 'meeting', 'offer', 'only', 'schedule', 'statement', 'time', 'today', 'your']


In [23]:
# CountVectorizer 객체는 문자열에 출현한 단어를 오름차순으로 정렬해 단어의 위치로 행렬을 만들어 리턴한다.
# 특정 단어가 출현할 경우 출현한 단어의 개수를 리턴하고 출현하지 않으면 0을 리턴한다.
# CountVectorizer 객체를 생성할 때 binary=True 옵션을 지정하면 같은 단어가 여러번 출현하더라도 무조건 1로 리턴한다.
cv = CountVectorizer(binary=True) # CountVectorizer 객체를 만든다.
x_train = cv.fit_transform(df_x) # 학습 후 적용
encoding_input = x_train.toarray() # 넘파이 타입의 배열로 데이터를 변환한다.
print(type(encoding_input))
print(encoding_input)

<class 'numpy.ndarray'>
[[0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0]
 [0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0]
 [1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1]]


위의 행렬에서 볼 수 있듯이 이메일 제목에서 총 17개의 단어가 발견되어 각 이메일 제목이 17개 크기의 벡터로 표현(인코딩)된 것을 확인할 수 있다.  
베르누이 나이브 베이즈에서 사용하기 위해 이메일 제목에 중복되서 출현한 단어가 있더라도 출현한 횟수로 표현되는 것이 아니라 단순히 1로 표현된 것을 알 수 있다.

In [25]:
# inverse_transform() 메소드로 고정된 크기의 벡터에 포함된 단어를 확인할 수 있다.
# print(cv.inverse_transform(encoding_input))
for s in cv.inverse_transform(encoding_input):
    print(s)

['free' 'game' 'only' 'today']
['cheapest' 'deal' 'flight']
['limited' 'offer' 'only' 'time' 'today']
['meeting' 'schedule' 'today']
['attached' 'flight' 'schedule' 'your']
['card' 'credit' 'statement' 'your']


In [28]:
# get_feature_names() 메소드로 고정된 벡터의 각 열(피쳐)이 어떤 단어를 의미하는지 확인할 수 있다.
print(cv.get_feature_names())

['attached', 'card', 'cheapest', 'credit', 'deal', 'flight', 'free', 'game', 'limited', 'meeting', 'offer', 'only', 'schedule', 'statement', 'time', 'today', 'your']


베르누이 나이브 베이즈 모델 학습하기  
사이킷런의 베르누이 나이브 베이즈 분류기는 기본적으로 라플라스 스무딩을 지원하므로 학습 데이터에 없던 단어가 테스트 데이터에 있어도 분류가 잘 진행된다.

라플라스 스무딩(Laplace Smoothing)  
0이라는 수는 곱셈과 나눗셈을 무력화시키는 값이므로 그 전에 아무리 의미있는 값이 도출된다 하더라도 마지막에 0을 곱해버리면 값은 0이 나오게 된다. 이런 경우가 상당히 빈번하기 때문에 0이 아닌 최소값으로 보정을 하게되는데 이를 라플라스 스무딩이라 한다.

In [33]:
df_y

0    1
1    1
2    1
3    0
4    0
5    0
Name: label, dtype: int64

In [34]:
bnb = BernoulliNB() # 베르누이 나이브 베이즈 모델을 만든다.
bnb.fit(x_train, df_y) # 베르누이 나이브 베이즈 모델을 학습시킨다.

BernoulliNB()

테스트 데이터를 다듬고 테스트 실행

In [35]:
test_email_list = [
    {'email title' : 'free flight offer', 'spam' : True},
    {'email title' : 'hey traveler free flight deal', 'spam' : True},
    {'email title' : 'limited free game offer', 'spam' : True},
    {'email title' : 'today flight schedule', 'spam' : False},
    {'email title' : 'your credit card attached', 'spam' : False},
    {'email title' : 'free credit card offer only today', 'spam' : False}
]
test_df = pd.DataFrame(test_email_list)
test_df['label'] = test_df.spam.map({True: 1, False: 0})
test_df

,email title,spam,label
0,free flight offer,True,1
1,hey traveler free flight deal,True,1
2,limited free game offer,True,1
3,today flight schedule,False,0
4,your credit card attached,False,0
5,free credit card offer only today,False,0


In [37]:
test_x = test_df['email title']
test_x

0                    free flight offer
1        hey traveler free flight deal
2              limited free game offer
3                today flight schedule
4            your credit card attached
5    free credit card offer only today
Name: email title, dtype: object

In [38]:
test_y = test_df['label']
test_y

0    1
1    1
2    1
3    0
4    0
5    0
Name: label, dtype: int64

In [39]:
# CountVectorizer 객체는 학습 데이터를 다듬을 때 이미 학습을 시켰으므로 테스트 시에는 적용만 시키면 된다.
x_test = cv.transform(test_x) # 적용

모델 테스트

In [40]:
predict = bnb.predict(x_test) # 학습 데이터로 학습한 결과에 따른 테스트 데이터의 예측값을 계산한다.
accuracy = accuracy_score(test_y, predict) #accuracy_score(실제값, 예측값)
print('정확도: {:.2%}'.format(accuracy))

정확도: 83.33%


In [41]:
conf_matrix = confusion_matrix(test_y, predict)
print(conf_matrix)

[[2 1]
 [0 3]]


In [42]:
print(classification_report(test_y, predict))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.75      1.00      0.86         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6



In [43]:
comparison = pd.DataFrame({'실제값': test_y, '예측값': predict})
comparison

,실제값,예측값
0,1,1
1,1,1
2,1,1
3,0,0
4,0,0
5,0,1
